In [ ]:
# csv 읽어오는 거 제한을 해제하는 코드
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
#주요 모듈 임포트
import pandas as pd

In [ ]:
#전자기기 리뷰 읽기
review_df = pd.read_csv('reviews_Electronics.csv', engine='python')
review_df

In [ ]:
#메타 데이터 읽기
meta_df = pd.read_csv('meta_ele.csv', engine = 'python')
meta_df

In [ ]:
#product list 호출
prod_df = pd.read_csv('product_list.csv',engine='python')
prod_ls = list(prod_df['product'])
#디지털 카메라 카테고리 걸러내는 함수
def cat_dslr(text):
    #해당 카테고리에 속하면 카테고리명을 수정하기 편하게 바꿈. 카테고리를 수정하고 싶으면 이곳에서 업데이트
    if text in ["[['Electronics', 'Camera & Photo', 'Digital Cameras', 'Point & Shoot Digital Cameras']]",
                "[['Electronics', 'Camera & Photo', 'Digital Cameras', 'Digital SLR Cameras']]",
                "[['Electronics', 'Camera & Photo', 'Digital Cameras']]",
                "[['Electronics', 'Camera & Photo', 'Digital Cameras', 'Compact System Cameras']]",
                "[['Electronics', 'Camera & Photo', 'Digital Cameras', 'Medium Format Digital Cameras']]"]:
        return 'ggggg' #ggggg로 값을 바꾸어 수정하기 쉽게 만듦
    return text

# asin code 기준으로 데이터 병합 함수
def asin_merge(review, meta):
    meta['categories'] = meta['categories'].apply(cat_dslr)
    digit_asin=meta[meta['categories'] == 'ggggg'].loc[:,['asin','title','description']]
    reviewNproduct = review.merge(digit_asin, on='asin')
    return reviewNproduct

#칼럼명 수정후 원하는 칼럼(product_name, review, review_date, score, user)만 빼오는 함수, 추가로 필요한 데이터 있으면 이거 이용
def recolumns(df):
    df.rename(columns={
        'title':'product_name',
        'reviewText':'review',
        'reviewTime':'review_date',
        'overall':'score',
        'reviewerName':'user',
    }, inplace=True)
    return df.loc[:, ['product_name','asin','review','review_date','score','user']]

#리뷰가 10개 미만인 제품일 경우 제거한다->asin 코드 기준으로 슬라이싱 및 리뷰 nan 값 제거
def slicing(df):
    cnt_df = pd.DataFrame(df['asin'].value_counts()).reset_index().rename(columns={'index':'asin','asin':'review_count'})
    cnt_df = cnt_df[cnt_df['review_count']>=10]
    new_df = df.merge(cnt_df, on='asin')
    del new_df['review_count']
    
    #nan 값 제거
    new_df=new_df.fillna(999)
    new_df=new_df[new_df['review']!=999]
    return new_df

#제품명을 모델명으로 줄임. 제품 리스트에 존재하지 않을 경우 그 데이터는 그냥 삭제 할 것
def product_modelize(text):

    for model in prod_ls:
        if " "+model+" " in text.lower():
            return model
    
    return '999'

In [ ]:
#함수 적용
digit_camera_review_df=asin_merge(review_df, meta_df)
new_df=recolumns(digit_camera_review_df)
new_df=slicing(new_df)
new_df['product_name']=new_df['product_name'].apply(product_modelize)

In [ ]:
newnew=new_df[new_df['product_name'] != '999']
newnew

In [ ]:
newnew.to_csv('camera_reviews.csv', encoding='utf-8', index=False)